In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def canny(img):

    canny = cv.Canny(img, 50, 150)
    return canny

In [3]:
def region_of_interest(img):
    height = img.shape[0]
    poly = np.array([[(200,height), (1200, height), (790,450), (580, 450)]])
    mask = np.zeros_like(img)
    cv.fillPoly(mask, poly, 255)
    return mask

In [4]:
def display_lines(img, lines):
    line_image = np.zeros_like(img)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line.reshape(4)
            cv.line(line_image, (x1,y1), (x2,y2), (0,255,0), 10)
    return line_image

In [5]:
def run_histogram_equalization(rgb_img):
    # convert from RGB color-space to YCrCb
    ycrcb_img = cv.cvtColor(rgb_img, cv.COLOR_BGR2YCrCb)

    # equalize the histogram of the Y channel
    ycrcb_img[:, :, 0] = cv.equalizeHist(ycrcb_img[:, :, 0])

    # convert back to RGB color-space from YCrCb
    equalized_img = cv.cvtColor(ycrcb_img, cv.COLOR_YCrCb2BGR)
    return equalized_img

In [6]:
def PerspectiveTransformm(scrPoints,dstPoints):
    M=cv.getPerspectiveTransform(scrPoints,dstPoints)
    Minv = cv.getPerspectiveTransform(dstPoints,scrPoints)
    
    return M,Minv

In [7]:
def warpPerspective(img,imgSize,M):
    return cv.warpPerspective(img,M,imgSize,cv.INTER_LINEAR)

In [8]:
def bird_view(img):
    src = np.array([[200,700],[580,450],[670,450],[1150,700]],np.float32)
    dst = np.array([[200,700], [200,50], [1000, 50], [1000, 700]],np.float32)
    M,Minv = PerspectiveTransformm(src,dst)
    warap =warpPerspective(img,img.shape[1::-1],M)
    return warap

In [ ]:
cap = cv.VideoCapture('project_video.mp4')
# Check if camera opened successfully
if (cap.isOpened()== False): 
  print("Error opening video stream or file")

while(cap.isOpened()):
  # Capture frame-by-frame
  ret, frame = cap.read()
  if ret == True:
    # Display the resulting frame

    
    hls = cv.cvtColor(frame, cv.COLOR_BGR2HLS)    
    low_yellow = np.uint8([ 10,   0, 100])
    up_yellow = np.uint8([ 40, 255, 255])
    mask = cv.inRange(hls, low_yellow, up_yellow)
    gray = cv.cvtColor(frame, cv.COLOR_RGB2GRAY)
    blur = cv.GaussianBlur(gray, (3,3), 0)
    interestt = blur & region_of_interest(blur)
    
    ret, thresh = cv.threshold(interestt, 210, 240, cv.THRESH_BINARY)
    
    new = mask | thresh
    
    interestt = new & region_of_interest(new)
    src=np.float32([[195,720],
                [590,460],
                [700,460],
                [1120,720]])
    dst=np.float32([[350,720],
                [410,0],
                [970,0],
                [1000,720]])
    M,Minv = PerspectiveTransformm(src,dst)
    warap =warpPerspective(interestt,interestt.shape[1::-1],M)
    
    new_canny = cv.Canny(warap, 50, 150)
    lines = cv.HoughLinesP(new_canny, 5, np.pi/180, 100, np.array([]), minLineLength=50, maxLineGap=200)
        

    line_img = display_lines(frame, lines)
    M,Minv = PerspectiveTransformm(dst,src)
    warap2 =warpPerspective(line_img,line_img.shape[1::-1],M)
     
    combo_image = cv.addWeighted(frame, 0.8, warap2, 1, 1)
    cv.imshow('Frame',combo_image)

        
    # Press Q on keyboard to  exit
    if cv.waitKey(25) & 0xFF == ord('q'):
      break

  else: 
    break

# When everything done, release the video capture object
cap.release()

# Closes all the frames
cv.destroyAllWindows()